In [1]:
from omnipy import Dataset, Model
import omnipy as om
om.setup_jupyter_ui()
om.runtime.config.root_log.log_to_stdout = False
om.runtime.config.data.ui.jupyter.color.style = om.DarkHighContrastTintedThemingBase16ColorStyles.GOTHAM_T16

ℹ️ 
 Omnipy has detected that it is being imported into a Jupyter
session. Please run the following in a separate cell:

 import omnipy as om
om.setup_jupyter_ui()

🌐 
 The Jupyter kernel currently runs in web browser mode. (If you are running Jupyter elsewhere, e.g. from within PyCharm, you probably want to restart the kernel.)

🔧 
 Workaround for Jupyter in PyCharm detection issue : immediately after kernel startup, run an empty or unrelated cell before the cell that loads Omnipy.

⚠️ 
 This cell was not loaded properly! You probably want to rerun the code cell above ( Click in the code cell, and press Shift+Enter ⇧ + ↩ ).

No Omnipy-related CSS styling or hidden widgets were loaded due to plain text output.

↻ 
 Make sure to rerun this cell whenever the kernel is restarted.

🔧 
 Workaround for JupyterLab erratic scrolling issue : set the "Windowing mode" setting under the "Notebook" section of the "Settings Editor" (from the "Settings" menu) to "defer" instead of the default "full" . Reactive functionality might be less responsive in this mode.

Cannot show ipywidgets in text

Cannot show ipywidgets in text

In [6]:
from nobel_api_output_model import NobelPrizesResult, LaureateBasic

# Omnipy tasks
@om.TaskTemplate()
def create_nobel_prize_urls(category: str | None, page_size: int, num_pages: int) -> om.HttpUrlDataset:
    def _nobel_price_api_urls(page_offset: int) -> om.HttpUrlModel:
        url = om.HttpUrlModel('https://api.nobelprize.org')
        url.path // '2.1' // 'nobelPrizes'
        if category:
            url.query['nobelPrizeCategory'] = category
        url.query['offset'] = page_offset
        url.query['limit'] = page_size
        return url

    return om.HttpUrlDataset({
        page_num * page_size: _nobel_price_api_urls(page_offset=page_num * page_size)
        for page_num in range(num_pages)
    })

@om.TaskTemplate()
async def fetch_and_extract_laureates_per_year(nobel_prize_urls: om.HttpUrlDataset) -> Dataset[Model[list[LaureateBasic]]]:
    from functools import reduce
    from operator import add

    per_page_nobel_prize_info = await Dataset[Model[NobelPrizesResult]].load(nobel_prize_urls)
    
    per_year_nobel_prizes = Dataset[Model[list[LaureateBasic]]]()
    for entry in (reduce(add, (_.nobelPrizes for _ in per_page_nobel_prize_info.values()))):
        if entry.laureates:
            per_year_nobel_prizes[str(entry.awardYear)] = entry.laureates
    
    return per_year_nobel_prizes


# Omnipy dataflows
@om.LinearFlowTemplate(
    create_nobel_prize_urls,
    fetch_and_extract_laureates_per_year,
)
def download_and_wrangle_nobel_prizes(category: str | None, page_size: int, num_pages: int):
    ...

# Run the dataflow and show results
literature_prizes = await download_and_wrangle_nobel_prizes.run(category='lit', page_size=25, num_pages=6)
literature_prizes

Cannot show ipywidgets in text

In [7]:
old_literature_prizes = literature_prizes
literature_prizes = om.JsonDataset()
literature_prizes.from_json(old_literature_prizes.to_json())

In [8]:
literature_prizes[-2:].peek(style='rose-pine-dawn-t16', bg=True, syntax='json', h_overflow='word_wrap')

Cannot show ipywidgets in text